***GENERATED CODE FOR routingdatapipeline230921 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DestinationState', 'transformation_label': 'String Indexer'}], 'feature': 'DestinationState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '314', 'mean': '', 'stddev': '', 'min': 'NJ', 'max': 'WA', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DestinationState'}, {'feature_label': 'DestinationState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DestinationState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationType', 'transformation_label': 'String Indexer'}], 'feature': 'LocationType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '314', 'mean': '', 'stddev': '', 'min': 'DC', 'max': 'Store', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationType'}, {'feature_label': 'LocationType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SourceState', 'transformation_label': 'String Indexer'}], 'feature': 'SourceState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '314', 'mean': '', 'stddev': '', 'min': 'KS', 'max': 'TX', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SourceState'}, {'feature_label': 'SourceState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SourceState')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'deliveryDate', 'transformation_label': 'Extract Date'}], 'feature': 'deliveryDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'deliveryDate'}, {'feature_label': 'deliveryDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('deliveryDate')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run routingdatapipeline230921Hooks.ipynb
try:
	#sourcePreExecutionHook()

	routingdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1632412317540_RoutingData_230921.csv', 'filename': '1632412317540_RoutingData_230921.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Biglynx_200921/RoutingData_230921.csv', 'viewFileName': 'RoutingData_230921.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run routingdatapipeline230921Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(routingdata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "OrderLineItemId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "314", "mean": "2908421.67", "stddev": "1044462.81", "min": "120001.0", "max": "4714271.0", "missing": "0"}, "updatedLabel": "OrderLineItemId"}, {"transformationsData": [{"feature_label": "DestinationState", "transformation_label": "String Indexer"}], "feature": "DestinationState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "314", "mean": "", "stddev": "", "min": "NJ", "max": "WA", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DestinationState"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DestinationZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "314", "mean": "62648.91", "stddev": "28574.53", "min": "7202", "max": "99201", "missing": "0"}, "updatedLabel": "DestinationZip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DestinationLat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "314", "mean": "33.33", "stddev": "4.21", "min": "29.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DestinationLat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DestinationLon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "314", "mean": "-93.69", "stddev": "12.02", "min": "-122.0", "max": "-73.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DestinationLon"}, {"transformationsData": [{"feature_label": "LocationType", "transformation_label": "String Indexer"}], "feature": "LocationType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "314", "mean": "", "stddev": "", "min": "DC", "max": "Store", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationType"}, {"transformationsData": [{"feature_label": "SourceState", "transformation_label": "String Indexer"}], "feature": "SourceState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "314", "mean": "", "stddev": "", "min": "KS", "max": "TX", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SourceState"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SourceZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "314", "mean": "69573.98", "stddev": "15385.5", "min": "7202", "max": "89506", "missing": "0"}, "updatedLabel": "SourceZip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SourceLat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "314", "mean": "33.15", "stddev": "3.65", "min": "29.0", "max": "40.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SourceLat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SourceLon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "314", "mean": "-95.63", "stddev": "8.7", "min": "-119.0", "max": "-74.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SourceLon"}, {"transformationsData": [{"feature_label": "deliveryDate", "transformation_label": "Extract Date"}], "feature": "deliveryDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "deliveryDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SourceToDestinationDistance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "314", "mean": "233.05", "stddev": "358.01", "min": "2.0", "max": "1794.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SourceToDestinationDistan..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "timeWindowGroup", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "314", "mean": "4.51", "stddev": "1.32", "min": "1", "max": "8", "missing": "0"}, "updatedLabel": "timeWindowGroup"}, {"feature": "DestinationState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "314", "mean": "0.48", "stddev": "0.88", "min": "0.0", "max": "6.0", "missing": "0"}, "updatedLabel": "DestinationState_stringin..."}, {"feature": "LocationType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "314", "mean": "0.27", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "LocationType_stringindexe..."}, {"feature": "SourceState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "314", "mean": "0.36", "stddev": "0.68", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "SourceState_stringindexer..."}, {"feature": "deliveryDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "314", "mean": "14.26", "stddev": "7.84", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "deliveryDate_dayofmonth"}, {"feature": "deliveryDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "314", "mean": "7.31", "stddev": "2.05", "min": "1", "max": "11", "missing": "0"}, "updatedLabel": "deliveryDate_month"}, {"feature": "deliveryDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "314", "mean": "2018.64", "stddev": "0.48", "min": "2018", "max": "2019", "missing": "0"}, "updatedLabel": "deliveryDate_year"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run routingdatapipeline230921Hooks.ipynb
try:
	#sinkPreExecutionHook()

	output = HDFSConnector.put(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/Data/output/RoutingData_UsingTimeWindow.csv', 'pathOnly': '/FileStore/platform/Data/output', 'filename': 'RoutingData_UsingTimeWindow', 'token': '', 'delimiter': ','}")
	#sinkPostExecutionHook(output)

except Exception as ex: 
	logging.error(ex)
